In [1]:
from _04_preprocessing import housing_prepared, housing_labels
from _04_preprocessing import full_pipeline
from _04_preprocessing import start_test_set

In [2]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'n_estimators': [3, 10, 30], "max_features": [2, 4, 6, 8]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
]

from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor()

grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                          scoring='neg_mean_squared_error',
                          return_train_score=True)
grid_search.fit(housing_prepared, housing_labels)

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid=[{'max_features': [2, 4, 6, 8],
                          'n_estimators': [3, 10, 30]},
                         {'bootstrap': [False], 'max_features': [2, 3, 4],
                          'n_estimators': [3, 10]}],
             return_train_score=True, scoring='neg_mean_squared_error')

수동으로 하이퍼파라미터를 조정할 수 있지만 사이킷런의 **GridSearchCV** 클래스를 사용할 수도 있음  
가능한 모든 조합에 대해 교차검증을 사용하여 평가함  
> **시간은 좀 걸리지만 최적의 조합을 찾을 수 있음**

In [3]:
grid_search.best_params_

{'max_features': 6, 'n_estimators': 30}

In [6]:
import numpy as np

cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

64452.557101401326 {'max_features': 2, 'n_estimators': 3}
55671.5991067315 {'max_features': 2, 'n_estimators': 10}
53139.07831104272 {'max_features': 2, 'n_estimators': 30}
61187.1007911414 {'max_features': 4, 'n_estimators': 3}
52425.780087676474 {'max_features': 4, 'n_estimators': 10}
50329.42359308412 {'max_features': 4, 'n_estimators': 30}
59186.79775493874 {'max_features': 6, 'n_estimators': 3}
52219.94852409528 {'max_features': 6, 'n_estimators': 10}
50053.77558253738 {'max_features': 6, 'n_estimators': 30}
59105.427696618885 {'max_features': 8, 'n_estimators': 3}
52201.993594673855 {'max_features': 8, 'n_estimators': 10}
50267.38072101483 {'max_features': 8, 'n_estimators': 30}
64073.06199783314 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
54474.05579952185 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
59751.71044365391 {'bootstrap': False, 'max_features': 3, 'n_estimators': 3}
52851.51185491938 {'bootstrap': False, 'max_features': 3, 'n_estimators': 

파라미터에 따른 점수와 최적의 파라미터를 추출할 수 있음  
  
> 더 나아가서 **데이터의 준비단계** 또한 하이퍼파라미터처럼 다룰 수 있음  
> - 예를 들어 이상치나 값이 빈 특성을 다룰 때 특성 선택을 자동으로 처리하는 데 사용할 수 있음

In [7]:
best_model = grid_search.best_estimator_

이런식으로 **가장 좋은 경우의 모델**을 가져올 수 있음

In [8]:
feature_importances = best_model.feature_importances_

In [9]:
feature_importances

array([8.92028199e-02, 6.99859320e-02, 4.01409065e-02, 1.85489183e-02,
       1.62099566e-02, 1.81138491e-02, 1.59920313e-02, 3.12416197e-01,
       7.28043991e-02, 1.03457261e-01, 7.05093063e-02, 1.60790946e-02,
       1.46553283e-01, 5.98995462e-05, 3.12985367e-03, 6.79629291e-03])

그리고 그 모델에 대해 학습에 사용된 **각 특성의 상대적인 중요도**를 알 수 있음

In [13]:
from _04_preprocessing import num_attribs
from pprint import pprint

extra_attribs = ["rooms_per_house", "pop_per_house", "bedrooms_per_room"]
cat_encoder = full_pipeline.named_transformers_["tf_cat"]
cat_one_hot_attribs = list(cat_encoder.categories_[0])
attribs = num_attribs + extra_attribs + cat_one_hot_attribs
pprint(sorted(zip(feature_importances, attribs), reverse=True))

[(0.3124161965575886, 'median_income'),
 (0.14655328269760712, 'INLAND'),
 (0.1034572609147763, 'pop_per_house'),
 (0.08920281987588922, 'longitude'),
 (0.07280439906846915, 'rooms_per_house'),
 (0.07050930630460514, 'bedrooms_per_room'),
 (0.06998593204718519, 'latitude'),
 (0.040140906518167596, 'housing_median_age'),
 (0.01854891825647091, 'total_rooms'),
 (0.01811384908845668, 'population'),
 (0.016209956639456607, 'total_bedrooms'),
 (0.01607909464419893, '<1H OCEAN'),
 (0.01599203126017584, 'households'),
 (0.006796292911805352, 'NEAR OCEAN'),
 (0.0031298536689226635, 'NEAR BAY'),
 (5.98995462246774e-05, 'ISLAND')]


각 중요도와 특성 이름을 같이 나타내는 예시  
> 이 중요도를 보고 **너무 낮은 중요도의 특성은 빼는 식으로** 튜닝할 수 있음